<a href="https://colab.research.google.com/github/megumihoshino/machine-learning-fundamental/blob/main/imageclassify_TF_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#1st step, checked out the last version of TF
import  tensorflow as tf
print(tf.__version__)

2.15.0


In [11]:
#preparing and downloading dataset
!wget --no-check-certificate \
  https://dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com/184/messy-vs-clean-room.zip \
  -O /tmp/messy_vs_clean_room.zip

--2024-04-30 13:18:25--  https://dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com/184/messy-vs-clean-room.zip
Resolving dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com (dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com)... 104.18.42.227, 172.64.145.29, 2606:4700:4400::6812:2ae3, ...
Connecting to dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com (dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com)|104.18.42.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Cookie coming from dicoding-academy-assets.sgp1.cdn.digitaloceanspaces.com attempted to set domain to digitaloceanspaces.com
Length: 70392746 (67M) [application/zip]
Saving to: ‘/tmp/messy_vs_clean_room.zip’

/tmp/messy_vs_clean 100%[===================>]  67.13M  73.7MB/s    in 0.9s    

2024-04-30 13:18:26 (73.7 MB/s) - ‘/tmp/messy_vs_clean_room.zip’ saved [70392746/70392746]



In [13]:
#extract file zip

import zipfile,os
local_zip = '/tmp/messy_vs_clean_room.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

In [14]:
'''tiap direktori data latih dan data validasi msing2 memiliki sub-direktori
clean dan messy. tiap sub-direktori menyimpan gbr yg sesuai dg nama sub-direktori
tsb. jd, pd sub-direktori 'clean' terdpt gbr2 ruangan yg rapi dan pd sub-direktori
'messy' terdpt gbr2 ruangan yg berantakan.'''

os.listdir('/tmp/images/train')

['clean', 'messy']

In [15]:
os.listdir('/tmp/images/val')

['clean', 'messy']

In [16]:
#APPLY IMAGE DATA GENERATOR FOR DATA TRAIN N DATA VALIDATION
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale= 1./255,
    rotation_range=20,
    horizontal_flip = True,
    shear_range = 0.2,
    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [17]:
#PREPARING DATA TRAIN AND VALIDATION (dari data yg di-load di dlm memori)
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
      validation_dir, #direktori data validasi
      target_size = (150,150), #resolusi seluruh gbr jd 150x150 pixel
      batch_size=4, #klasifikasi 2 kelas, pke class_mode = 'binary'
      class_mode = 'binary')


Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [21]:
#MEMBANGUN MODEL CNN (ada 4 lapis layer konvolusi dan max pooling)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation ='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])


In [20]:
#MENAMPILKAN SUMMARY DARI ARSITEKTUR YG TELAH DIBUAT
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 17, 17, 128)       0